In [ ]:
%%capture
%pip install pandas
%pip install scikit-learn
%pip install numpy
%pip install sentence-transformers
%pip install tf-keras
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sentence_transformers import SentenceTransformer

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for tf-keras from https://files.pythonhosted.org/packages/8a/ed/e08afca471299b04a34cd548e64e89d0153eda0e6cf9b715356777e24774/tf_keras-2.18.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Load initial dataset
initial_accions_df = pd.read_csv('./datasets/accions.csv')
initial_accions_df.drop(['Usuari', 'Representat'], axis=1, inplace=True)
initial_tramits_df = pd.read_csv('./datasets/tramits.csv')
tramits_map = initial_tramits_df.set_index('Id')['Titol'].to_dict()
initial_tramits_df.drop(['Titol'], axis=1, inplace=True)
merged_df = initial_accions_df.merge(initial_tramits_df, left_on='Tramit', right_on='Id').drop(['Id'], axis=1)


In [ ]:
# Actions by session (deprecated)
session_counts = initial_df.groupby('Sessio').size()
count_by_rows = session_counts.value_counts().sort_index()
result = pd.DataFrame({
    'row_count': [1, 2, 3, 4, '>5'],
    'num_sessions': [
        count_by_rows.get(1, 0),
        count_by_rows.get(2, 0),
        count_by_rows.get(3, 0),
        count_by_rows.get(4, 0),
        count_by_rows[count_by_rows.index > 5].sum()
    ]
})
result

In [ ]:
# Removed single-action sessions (deprecated)
session_counts = initial_df.groupby('Sessio').size().reset_index(name='count')
sessions_to_keep = session_counts[session_counts['count'] > 1]
df = initial_df[initial_df['Sessio'].isin(sessions_to_keep['Sessio'])]

In [6]:
# Remove non-vigent
initial_df = merged_df[merged_df['Vigent']].drop(columns=['Vigent'])


In [7]:
# Encode action values
df = initial_df.copy()  # If df is a subset of another dataframe, make an explicit copy first
df['Accio_Tramit'] = df['Accio'] + '_' + df['Tramit']
label_encoder = LabelEncoder()
df['action_id'] = label_encoder.fit_transform(df['Accio_Tramit'])
df.drop(['Accio', 'Tramit', 'Accio_Tramit'], axis=1, inplace=True)
df = df.sort_values(by=['Sessio', 'Data'])

In [8]:
# Remove repeated actions
df = df.loc[
    df['action_id'] != df.groupby('Sessio')['action_id'].shift()
]

In [9]:
# Remove sessions with less than n actions
n = 4
session_counts = df.groupby('Sessio').size().reset_index(name='count')
sessions_to_keep = session_counts[session_counts['count'] >= n]
df = df[df['Sessio'].isin(sessions_to_keep['Sessio'])]

In [10]:
# Store sequences in a dictionary in order
df_sorted = df.sort_values(by=['Sessio', 'Data'])
session_sequences = {}
for session_id, group in df_sorted.groupby('Sessio'):
    action_sequence = group['action_id'].tolist()
    session_sequences[session_id] = action_sequence

In [ ]:
# Generate sequences embeddings (deprecated)
num_actions = len(label_encoder.classes_)
embedding_dim = 50
embedding_map = {}
for action_id in range(num_actions):
    random_embedding = np.random.randn(embedding_dim).astype(np.float32)
    embedding_map[action_id] = random_embedding
sequence_data = []
for session_id, action_sequence in session_sequences.items():
    for i in range(len(action_sequence) - 3):
        input1 = embedding_map[action_sequence[i]]
        input2 = embedding_map[action_sequence[i + 1]]
        input3 = embedding_map[action_sequence[i + 2]]
        label = embedding_map[action_sequence[i + 3]]
        sequence_data.append((input1, input2, input3, label))

In [16]:
embedding_map = {}
embedding_dim = 384
def get_embedding(sentence):
    return embedding_model.encode(sentence)
action_type_map = {
    'AFIT': 'Acces a la fitxa informativa de ',
    'AFST': 'Acces a la fitxa de solicitud de ',
    'PFST': 'Presentacio del formulari de solicitud de ',
}
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
for indx, action in enumerate(label_encoder.classes_):
    action_type = action.split('_')[0]
    tramit_id = action.split('_')[1]
    action_text = action_type_map[action_type] + tramits_map[tramit_id]
    embedding_map[indx] = get_embedding(action_text)
sequence_data = []
for session_id, action_sequence in session_sequences.items():
    for i in range(len(action_sequence) - 3):
        input1 = embedding_map[action_sequence[i]]
        input2 = embedding_map[action_sequence[i + 1]]
        input3 = embedding_map[action_sequence[i + 2]]
        label = embedding_map[action_sequence[i + 3]]
        sequence_data.append((input1, input2, input3, label))

384

In [ ]:
# Generate sequences + label one-hot (deprecated)
num_actions = len(label_encoder.classes_)
def one_hot_encode(action_id, num_classes):
    one_hot_vector = np.zeros(num_classes, dtype=np.int8)
    one_hot_vector[action_id] = 1
    return one_hot_vector
sequence_data = []

for session_id, action_sequence in session_sequences.items():
    for i in range(len(action_sequence) - 3):
        input1 = one_hot_encode(action_sequence[i], num_actions)
        input2 = one_hot_encode(action_sequence[i + 1], num_actions)
        input3 = one_hot_encode(action_sequence[i + 2], num_actions)
        label = one_hot_encode(action_sequence[i + 3], num_actions)
        sequence_data.append((input1, input2, input3, label))

In [ ]:
print(len(sequence_data))
print(sequence_data[0])